In [1]:
import os
import librosa
import numpy as np
import soundfile as sf

input_dir = r"D:\Project\Data\Processed\cough"
output_dir = r"D:\Project\Data\audio\cough"
os.makedirs(output_dir, exist_ok=True)

def add_white_noise(audio, noise_level=0.005):
    noise = np.random.normal(0, noise_level, audio.shape)
    return audio + noise

for fname in os.listdir(input_dir):
    if fname.endswith(".wav"):
        path = os.path.join(input_dir, fname)
        y, sr = librosa.load(path, sr=None)

        # Add noise
        y_noisy = add_white_noise(y)

        # Clip to avoid overflow
        y_noisy = np.clip(y_noisy, -1.0, 1.0)

        # Save
        out_path = os.path.join(output_dir, f"noisy_{fname}")
        sf.write(out_path, y_noisy, sr)

print("Noise-augmented cough samples saved.")


Noise-augmented cough samples saved.


In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
import random

input_dir = r"D:\Project\Data\Processed\sneeze\sneeze"
output_dir = r"D:\Project\Data\audio\sneeze"
os.makedirs(output_dir, exist_ok=True)

def add_noise(data, noise_level=0.003):
    noise = np.random.randn(len(data))
    return data + noise_level * noise

def pitch_shift(data, sr, n_steps=2):
    return librosa.effects.pitch_shift(data, sr=sr, n_steps=n_steps)

def time_stretch(data, rate=1.1):
    return librosa.effects.time_stretch(data, rate=rate)

def change_volume(data, gain=1.2):
    return data * gain

augmentations_per_file = 3

for fname in os.listdir(input_dir):
    if fname.endswith(".wav"):
        path = os.path.join(input_dir, fname)
        y, sr = librosa.load(path, sr=16000)

        for i in range(augmentations_per_file):
            y_aug = y.copy()

            # Randomly apply 2 augmentations
            if random.choice([True, False]):
                y_aug = pitch_shift(y_aug, sr, n_steps=random.choice([-1, 1, 2]))
            if random.choice([True, False]):
                y_aug = time_stretch(y_aug, rate=random.uniform(0.9, 1.1))
            if random.choice([True, False]):
                y_aug = change_volume(y_aug, gain=random.uniform(0.8, 1.2))
            if random.choice([True, False]):
                y_aug = add_noise(y_aug, noise_level=random.uniform(0.002, 0.01))

            y_aug = np.clip(y_aug, -1.0, 1.0)

            out_fname = f"aug_{i}_{fname}"
            sf.write(os.path.join(output_dir, out_fname), y_aug, sr)

print(" Sneeze augmentation completed.")


✅ Sneeze augmentation completed.


In [4]:
import os
import librosa
import soundfile as sf
import numpy as np
from scipy.signal import butter, lfilter
from pedalboard import Pedalboard, Reverb

# Paths
input_dir = r"D:\Project\Data\Processed\sneeze\sneeze"
output_dir = r"D:\Project\Data\audio\sneeze"
os.makedirs(output_dir, exist_ok=True)

# Low-pass filter
def low_pass_filter(y, sr, cutoff=3000, order=6):
    nyq = 0.5 * sr
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return lfilter(b, a, y)

# Augmentation functions
def add_reverb(y, sr):
    board = Pedalboard([Reverb(room_size=0.75)])
    return board(y, sr)

# Process files
for filename in os.listdir(input_dir):
    if filename.endswith(".wav"):
        path = os.path.join(input_dir, filename)
        y, sr = librosa.load(path, sr=16000)

        # Reverb
        y_reverb = add_reverb(y, sr)
        sf.write(os.path.join(output_dir, filename.replace(".wav", "_reverb.wav")), y_reverb, sr)

        # Low-pass filter
        y_lowpass = low_pass_filter(y, sr)
        sf.write(os.path.join(output_dir, filename.replace(".wav", "_lowpass.wav")), y_lowpass, sr)


In [ ]:
import os
import librosa
import soundfile as sf

input_dir = r"D:\Project\Data\Processed\sneeze\sneeze"
output_dir = r"D:\Project\Data\audio\sneeze"
os.makedirs(output_dir, exist_ok=True)

# Define pitch shifts in semitones
pitch_shifts = [-2, 2, 3]

for fname in os.listdir(input_dir):
    if fname.endswith(".wav"):
        path = os.path.join(input_dir, fname)
        y, sr = librosa.load(path, sr=16000)

        for shift in pitch_shifts:
            y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=shift)
            y_shifted = y_shifted[:sr*2]  # Ensure 2 sec max
            y_shifted = y_shifted / max(abs(y_shifted))  # Normalize
            out_fname = f"{os.path.splitext(fname)[0]}_pitch{shift:+}.wav"
            sf.write(os.path.join(output_dir, out_fname), y_shifted, sr)

print(" Pitch-shift augmentation complete.")


✅ Pitch-shift augmentation complete.
